In [2]:
!pip install confluent_kafka

     |████████████████████████████████| 8.1 MB 3.2 MB/s eta 0:00:01


In [3]:
!pip install avro

     |████████████████████████████████| 67 kB 397 kB/s eta 0:00:01
  Created wheel for avro: filename=avro-1.10.0-py3-none-any.whl size=96740 sha256=8aad8de239de37aeba73da6301731281701aed2ed93f64cdff7238ce4a50e456
  Stored in directory: /home/jovyan/.cache/pip/wheels/e6/b6/4c/5fdfc1ad8cdc72eaca148e328fed1f19a9e67ac7303e1546c7
Successfully built avro


In [4]:
from confluent_kafka import Producer
import sys


# Producer

In [5]:
conf = {'bootstrap.servers': "kafka1:9092"}

In [6]:
p = Producer(**conf)

### Delivery Call-Back 
Optional per-message delivery callback (triggered by poll() or flush()) when a message has been successfully delivered or permanently
failed delivery (after retries).

In [7]:
def delivery_callback(err, msg):
        if err:
            sys.stderr.write('%% Message failed delivery: %s\n' % err)
        else:
            sys.stderr.write('%% Message delivered to %s [%d] @ %d\n' %
                             (msg.topic(), msg.partition(), msg.offset()))

In [8]:
lines = ["ciao ciao ciao", "a b c", "hey hey hey"]
topic = "test"

In [10]:
for line in lines:
    try:
        # Produce line (without newline)
        print(line)
        p.produce(topic, line.rstrip(), callback=delivery_callback)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

ciao ciao ciao
a b c
hey hey hey


<ipython-input-10-b79be27ed7b6>:5: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  p.produce(topic, line.rstrip(), callback=delivery_callback)


### NOTE
Since produce() is an asynchronous API this poll() call
will most likely not serve the delivery callback for the
last produce()d message.

In [11]:
p.poll(0)

% Message delivered to test [0] @ 0
% Message delivered to test [0] @ 1
% Message delivered to test [0] @ 2
% Message delivered to test [0] @ 3
% Message delivered to test [0] @ 4
% Message delivered to test [0] @ 5


2

In [12]:
sys.stderr.write('%% Waiting for %d deliveries\n' % len(p))

% Waiting for 0 deliveries


In [13]:
p.flush()

0

# Consumer

In [14]:
from confluent_kafka import Consumer, KafkaException
import sys
import getopt
import json
import logging
from pprint import pformat

In [84]:
conf = {
    'bootstrap.servers': "kafka1:9092", 
    'group.id': "mygroup", 
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'
}

In [18]:
# Create logger for consumer (logs will be emitted when poll() is called)
logger = logging.getLogger('consumer')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)-15s %(levelname)-8s %(message)s'))
logger.addHandler(handler)

In [19]:
 c = Consumer(conf, logger=logger)

In [20]:
def print_assignment(consumer, partitions):
        print('Assignment:', partitions)

In [21]:
topics = [topic]

In [22]:
# Subscribe to topics
c.subscribe(topics, on_assign=print_assignment)

In [23]:
# Read messages from Kafka, print to stdout
try:
    while True:
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            # Proper message
            sys.stderr.write('%% %s [%d] at offset %d with key %s:\n' %
             (msg.topic(), msg.partition(), msg.offset(), str(msg.key())))
            print(msg.value())
except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')
finally:
    # Close down consumer to commit final offsets.
    c.close()

Assignment: [TopicPartition{topic=test,partition=0,offset=-1001,error=None}]
b'ciao ciao ciao'
b'a b c'
b'hey hey hey'
b'ciao ciao ciao'
b'a b c'
b'hey hey hey'


% test [0] at offset 0 with key None:
% test [0] at offset 1 with key None:
% test [0] at offset 2 with key None:
% test [0] at offset 3 with key None:
% test [0] at offset 4 with key None:
% test [0] at offset 5 with key None:
% test [0] at offset 6 with key None:
% test [0] at offset 7 with key None:
% test [0] at offset 8 with key None:
% test [0] at offset 9 with key None:
% test [0] at offset 10 with key None:
% test [0] at offset 11 with key None:
% test [0] at offset 12 with key None:
% test [0] at offset 13 with key None:
% test [0] at offset 14 with key None:


b'Dolor neque velit sed voluptatem etincidunt velit.'
b'Dolor ipsum dolor numquam dolorem tempora numquam.'
b'Amet non quaerat ut quaerat porro.'
b'Numquam tempora tempora voluptatem est amet.'
b'Quiquia amet magnam etincidunt sit.'
b'Magnam ipsum ut dolorem.'
b'Neque sit voluptatem porro.'
b'Porro quaerat porro quaerat.'
b'Dolor dolore quiquia velit neque.'


%% Aborted by user


Let's try to copy the notebook

# Admin API

In [24]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions, ConfigResource, ConfigSource
from confluent_kafka import KafkaException
import sys
import threading
import logging

In [25]:
logging.basicConfig()


In [42]:
conf = {'bootstrap.servers': "kafka1:9092,kafka2:9093"}

In [43]:
a = AdminClient(conf)

### Creating Topics

In [48]:
topic_names = ["topic4", "topic2", "topci3"]

In [49]:
new_topics = [NewTopic(topic, num_partitions=3, 
        replication_factor=1) for topic in topic_names]

In [50]:
fs = a.create_topics(new_topics)

In [51]:
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} created".format(topic))
    except Exception as e:
        print("Failed to create topic {}: {}".format(topic, e))

Topic topic4 created
Failed to create topic topic2: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'topic2' already exists."}
Failed to create topic topci3: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'topci3' already exists."}


In [67]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 6 topics:
  "test" with 1 partition(s)
  "_schemas" with 1 partition(s)
  "topic1" with 4 partition(s)
  "__consumer_offsets" with 50 partition(s)
  "_confluent-metrics" with 12 partition(s)
  "__confluent.support.metrics" with 1 partition(s)


### Deleting Topics

In [52]:
fs = a.delete_topics(topic_names, operation_timeout=30)

In [53]:
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} deleted".format(topic))
    except Exception as e:
        print("Failed to delete topic {}: {}".format(topic, e))

Topic topic4 deleted
Topic topic2 deleted
Topic topci3 deleted


### Creating Partitions

In [70]:
topic = "topic1"

In [71]:
new_parts = [NewPartitions(topic, int(5))]

In [72]:
fs = a.create_partitions(new_parts, validate_only=False)
# Wait for operation to finish.
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Additional partitions created for topic {}".format(topic))
    except Exception as e:
        print("Failed to add partitions to topic {}: {}".format(topic, e))

Failed to add partitions to topic topic1: KafkaError{code=INVALID_PARTITIONS,val=37,str="Topic already has 5 partitions."}


In [83]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if str(t)==topic:
        for p in iter(t.partitions.values()):
            if p.error is not None:
                errstr = ": {}".format(p.error)
            else:
                errstr = ""
            print("partition {} leader: {}, replicas: {}," 
                  " isrs: {} errstr: {}".format(p.id, p.leader, p.replicas, p.isrs, errstr))

 6 topics:
partition 0 leader: 1, replicas: [1], isrs: [1] errstr: 
partition 1 leader: 2, replicas: [2], isrs: [2] errstr: 
partition 2 leader: 1, replicas: [1], isrs: [1] errstr: 
partition 3 leader: 2, replicas: [2], isrs: [2] errstr: 
partition 4 leader: 1, replicas: [1], isrs: [1] errstr: 


IndentationError: unexpected indent (<ipython-input-86-e4032d168f3d>, line 7)